### Exploring DBSCAN, K-Means, & Agglomerative Clustering to find the optimum number of clusters

In [2]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.cluster import silhouette_score

In [3]:
# need version 1.4.1 to get read_pickle method to work
pd.__version__

'1.4.2'

In [5]:
# get location from clean pickel file
ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
cln_pkl_loc = os.path.join(ROOT_DIR, 'data','cleanweathersmall.pkl')
# load it onto df
df = pd.read_pickle(cln_pkl_loc)
# quick look at the df
df.sample(3)

,station,time,temp,dwpt,rhum,prcp,wdir,wspd,pres
846433,72650,2022-01-24 01:00:00,-9.8,-11.8,85.0,0.1,157.0,20.2,1012.3
2261520,KHZY0,2022-06-05 00:00:00,15.0,6.0,55.0,0.0,50.0,5.4,1016.2
1243198,KAHQ0,2022-05-05 22:00:00,11.8,11.8,100.0,8.2,40.0,16.6,1012.0


### Collapse All Features -- Use Average

## Apply DBSCAN Algorithm to create clusters

### Create clusters by grouping all stations by month and by hours based on datestamps


In [6]:
# loop thru all twelve months:
DBSCAN_list = [] # = pd.DataFrame()
for mth in range(1,13):
    # loop thru all 24 hours
    for hr in range(24):
        # query df based on month and time
        tmp_df = df.query(f'time.dt.hour == {hr} and time.dt.month == {mth}')
        # remove time variable from df
        tmp_df = tmp_df.loc[: , tmp_df.columns != 'time']
        # calculate mean of such datafrme
        tmp_df = tmp_df.groupby('station').mean()
        # drop null values
        tmp_df.dropna(inplace=True)
        # if we get a result, calculate DBSCAN clusters
        if not tmp_df.empty:
            # rescale the data to zero mean and unit variance
            scaler = StandardScaler()
            scaler.fit(tmp_df)
            X_scaled = scaler.transform(tmp_df)
            # create the DBSCAN object; use all processors available
            dbscan = DBSCAN(n_jobs=-1)
            # run the DBSCAN algorithm
            clusters = dbscan.fit_predict(X_scaled)
            # attache clusters to df
            tmp_df['DBSCAN_cluster'] = clusters
            # add month and hour as variables
            tmp_df['hr'], tmp_df['mth'] = hr, mth
            # calculate the silhouette score; only if they're not outliers
            tmp_df['silhouette_score'] = -1
            if set(clusters) != {-1}:
                tmp_df['silhouette_score'] = silhouette_score(X_scaled, clusters)                
            # collect all dfs
            DBSCAN_list.append(tmp_df)
            
# collect all DBSCAN dfs into one
DBSCAN_df = pd.concat(DBSCAN_list)
# reset the index; we can query the 'station' as a column much easier
DBSCAN_df.reset_index(inplace=True)

### Select Clusters Where AA Belongs to

In [7]:
# list the clusters where KARB0 shows up; exclude the -1 (outliers)
AA_Clusters = DBSCAN_df.query('station == "KARB0" and DBSCAN_cluster != -1')['DBSCAN_cluster'].unique()

In [8]:
AA_Clusters

array([4, 1, 3, 7, 2, 0, 6], dtype=int64)

In [10]:
# what stations make up these clusters?
AA_near_stations = DBSCAN_df.query('DBSCAN_cluster in @AA_Clusters and silhouette_score > 0')['station'].unique()
len(AA_near_stations), AA_near_stations

(23,
 array(['6URQB', 'DCBG8', 'HEZQT', 'KFGN0', 'MUKMN', 'NCUQS', '4DUJO',
        '9H92X', 'ATA0X', 'CTGT0', 'MAU7O', '71273', '71667', 'CXQT0',
        'L1YUU', 'LII7V', '71147', '71564', '71962', '8ZB0I', 'SJZBK',
        '71849', 'GCHAU'], dtype=object))

In [8]:
# export df with clusters; we'll use these results for K-Means
#DBSCAN_df.to_pickle('DBSCAN_clusters.pkl')

In [11]:
station_loc = os.path.join(ROOT_DIR, 'station_analysis','AAstation.csv') #station data to get distance for comparision
dfstations=pd.read_csv(station_loc)

In [16]:
dfstations

,id,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,milesfromstat
0,0CNUO,Rawson Lake,CA,ON,NaN,NaN,49.6500,-93.7200,358.0,America/Rainy_River,NaN,NaN,2000-01-01,2022-08-10,2000-01-01,2022-01-01,701.767667
1,0CO7B,Kingsville Moe,CA,ON,NaN,NaN,42.0400,-82.6700,200.0,America/Toronto,NaN,NaN,2000-01-01,2022-09-17,2000-01-01,2021-01-01,56.683564
2,0FV1F,Tillsonburg Wwtp,CA,ON,NaN,NaN,42.8600,-80.7200,213.0,America/Toronto,NaN,NaN,2000-01-01,2022-09-18,2000-01-01,2021-01-01,160.560157
3,13178,Jamestown,CA,ON,NaN,NaN,43.8000,-81.1800,319.0,America/Toronto,NaN,NaN,2006-04-01,2015-07-04,2006-01-01,2015-01-01,169.519759
4,1J1PJ,Gore Bay,CA,ON,71956.0,NaN,45.8800,-82.5700,189.0,America/Nipigon,2020-01-01,2022-09-21,2010-07-09,2022-09-18,2010-01-01,2022-01-01,259.177215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,ZFZUV,Winnipeg Intl Airport,CA,MB,NaN,NaN,49.9100,-97.2400,239.0,America/Winnipeg,2020-01-01,2022-09-21,2018-10-30,2022-09-19,2018-01-01,2021-01-01,836.102103
628,ZNWZW,Columbus Municipal Airport,US,NE,NaN,KOLU,41.4500,-97.3333,440.0,America/Chicago,2022-04-23,2022-09-22,2022-04-24,2022-04-24,NaN,NaN,702.574295
629,ZUQJS,Ephraim-Gibraltar Airport,US,WI,NaN,K3D2,45.1357,-87.1881,234.0,America/Menominee,2022-05-06,2022-09-22,NaN,NaN,NaN,NaN,264.877147
630,ZWC6W,Kapuskasing Airport,CA,ON,NaN,NaN,49.4100,-82.4700,226.0,America/Toronto,2020-01-01,2022-09-21,2018-10-29,2022-09-19,2018-01-01,2022-01-01,500.149893


In [17]:
dfstations.merge(pd.DataFrame(AA_near_stations,columns=['station']),how='inner', left_on='id',right_on='station')

,id,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,milesfromstat,station
0,4DUJO,Fergus Shand Dam,CA,ON,NaN,NaN,43.7300,-80.3300,418.0,America/Toronto,NaN,NaN,2000-01-01,2022-09-18,2000-01-01,2022-01-01,201.949443,4DUJO
1,6URQB,Portage La Prairie CDA,CA,MB,NaN,NaN,49.9500,-98.2700,259.0,America/Winnipeg,NaN,NaN,2000-01-01,2022-09-16,2000-01-01,2022-01-01,876.102820,6URQB
2,71147,Carman U Of M CS,CA,MB,71147.0,CWNK,49.5000,-98.0300,268.0,America/Winnipeg,1996-11-01,2022-09-22,1995-06-27,2022-09-18,1995-01-01,2022-01-01,850.726151,71147
3,71273,Belle River,CA,SK,NaN,KWDD,42.3000,-82.7000,184.0,America/Toronto,1994-12-01,2022-08-23,1995-01-07,2022-09-18,2009-01-01,2022-01-01,53.870113,71273
4,71564,Morden CDA CS,CA,MB,71564.0,CXMD,49.1900,-98.0800,298.0,America/Winnipeg,2001-09-16,2022-09-22,1999-12-01,2022-09-18,1999-01-01,2022-01-01,841.929778,71564
5,71667,Thunder Bay CS,CA,ON,71667.0,CZTB,48.3700,-89.3300,199.0,America/Thunder_Bay,2005-01-01,2022-09-22,2003-10-01,2022-09-15,2003-01-01,2022-01-01,503.946973,71667
6,71849,Winnipeg Airport CS,CA,MB,71849.0,NaN,49.9200,-97.2500,239.0,America/Winnipeg,2001-09-16,2022-09-20,1996-10-01,2022-09-17,1996-01-01,2022-01-01,836.860088,71849
7,71962,Fort Frances RCS,CA,ON,71962.0,CTAG,48.6500,-93.4400,342.0,America/Winnipeg,2008-04-29,2022-09-20,2007-10-08,2022-09-18,2007-01-01,2022-01-01,646.477980,71962
8,8ZB0I,Kleefeld (mafri),CA,MB,71646.0,NaN,49.5200,-96.9000,246.0,America/Winnipeg,2020-01-01,2022-09-21,2014-09-17,2022-09-19,2014-01-01,2022-01-01,808.417487,8ZB0I
9,9H92X,Fergus Moe,CA,ON,NaN,NaN,43.7000,-80.3800,396.0,America/Toronto,NaN,NaN,2000-01-01,2022-08-15,2000-01-01,2019-01-01,198.747756,9H92X
